In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import os
import re
import json
import requests as req
from bs4 import BeautifulSoup as bs

my_options = webdriver.ChromeOptions()

#最大化視窗
my_options.add_argument("--start-maximized")
#開啟無痕模式    
my_options.add_argument("--incognito")
#禁用彈出攔截
my_options.add_argument("--disable-popup-blocking")
#取消通知
my_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=my_options)

In [14]:
url ='https://www.gutenberg.org/browse/languages/zh'

# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 

# 建立 list 來放置列表資訊
list_posts = []

In [15]:
list_posts.clear()
NumberOfNode=0

folderPath = 'test Gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 取得 列表 的文字與超連結
for a in soup.select('div.pgdbbylanguage li.pgdbetext a[href]'):
    regex01 = r'[\u4E00-\u9FFF]+(?!\s|\r|<br>)'
    bookTitle=a.get_text()
    bookHref=a['href']
    match01 = re.match(regex01, bookTitle)
    regex02 = r"\/ebooks\/"
    bookNo = re.sub(regex02, "", bookHref)
    if match01 and "臺灣通史" not in a.get_text():
        # print(a.get_text())
        # print(bookNo) # 或是 a.get('href') 
    
        # 加入列表資訊
        list_posts.append({
            'title': bookTitle,
            'id': bookNo
    })

for book in list_posts: 
    driver.get(f'https://www.gutenberg.org/cache/epub/{book["id"]}/pg{book["id"]}.txt')
    file_path =os.path.join(f"{folderPath}/{book['title']}.txt")
    text = driver.find_element(By.CSS_SELECTOR,'pre').get_attribute('innerText')
    with open(file_path,'w',encoding='utf-8') as f:
        f.write(text)

    chineseText = []
    onlyChineseRegex = r"[^\S\r\n]*[\u4e00-\u9fa5]+[^\S\r\n]*"
    with open(file_path,'r',encoding='utf-8') as f:
        for i in f.readlines():
            if re.match(onlyChineseRegex,i):
                chineseText.append(i)

    with open(file_path, 'w', encoding='utf-8') as f:
        for text in chineseText:
            f.write(text)
    
    NumberOfNode +=1

print(f'總本數為{NumberOfNode}本')



總本數為477本
